# Using traditional models and feature engineering to classify SHL timeseries

In [1]:
import pandas as pd

location = 'shl-dataset/train/Hand/'

acc_x = pd.read_csv(location + 'Acc_x.txt', header=None, sep=' ').to_numpy()
print('Acc_x Import Done')
acc_y = pd.read_csv(location + 'Acc_y.txt', header=None, sep=' ').to_numpy()
print('Acc_y Import Done')
acc_z = pd.read_csv(location + 'Acc_z.txt', header=None, sep=' ').to_numpy()
print('Acc_z Import Done')
mag_x = pd.read_csv(location + 'Mag_x.txt', header=None, sep=' ').to_numpy()
print('Mag_x Import Done')
mag_y = pd.read_csv(location + 'Mag_y.txt', header=None, sep=' ').to_numpy()
print('Mag_y Import Done')
mag_z = pd.read_csv(location + 'Mag_z.txt', header=None, sep=' ').to_numpy()
print('Mag_z Import Done')
gyr_x = pd.read_csv(location + 'Gyr_x.txt', header=None, sep=' ').to_numpy()
print('Gyr_x Import Done')
gyr_y = pd.read_csv(location + 'Gyr_y.txt', header=None, sep=' ').to_numpy()
print('Gyr_y Import Done')
gyr_z = pd.read_csv(location + 'Gyr_z.txt', header=None, sep=' ').to_numpy()
print('Gyr_z Import Done')

y = pd.read_csv(location + 'Label.txt', header=None, sep=' ').mode(axis=1).to_numpy().flatten()
print('Label Import Done')

Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Label Import Done


In [2]:
import numpy as np

from scipy import signal
from scipy.special import entr


def magnitude(x,y,z):
  return np.sqrt(x**2 + y**2 + z**2)

def entrop(pk,axis=0):
     pk = pk / np.sum(pk, axis=axis, keepdims=True)
     vec = entr(pk)
     S = np.sum(vec, axis=axis)
     return S

def autocorr(x,axis=0):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]


# Magnitude Calculation
acc_MAG = magnitude(acc_x,acc_y,acc_z)
mag_MAG = magnitude(mag_x,mag_y,mag_z)
gyr_MAG = magnitude(gyr_x,gyr_y,gyr_z)

# Statistical Feature Calculation
acc_mean = np.mean(acc_MAG,axis=1)
acc_std = np.std(acc_MAG,axis=1)
acc_max = np.max(acc_MAG,axis=1)
acc_min = np.min(acc_MAG,axis=1)
mag_mean = np.mean(mag_MAG,axis=1)
mag_std = np.std(mag_MAG,axis=1)
mag_max = np.max(mag_MAG,axis=1)
mag_min = np.min(mag_MAG,axis=1)
gyr_mean = np.mean(gyr_MAG,axis=1)
gyr_std = np.std(gyr_MAG,axis=1)
gyr_max = np.max(gyr_MAG,axis=1)
gyr_min = np.min(gyr_MAG,axis=1)

# Frequency Domain Feature Calculation
fs = 100
acc_FREQ,acc_PSD = signal.welch(acc_MAG,fs,nperseg=500,axis=1)
mag_FREQ,mag_PSD = signal.welch(mag_MAG,fs,nperseg=500,axis=1)

# Max PSD value
acc_PSDmax = np.max(acc_PSD,axis=1)
mag_PSDmax = np.max(mag_PSD,axis=1)

# Frequency Entropy
acc_entropy = entrop(acc_PSD,axis=1)
mag_entropy = entrop(mag_PSD,axis=1)

# Frequency Center
acc_fc = np.sum((acc_FREQ*acc_PSD),axis=1) / np.sum(acc_PSD,axis=1)
mag_fc = np.sum((mag_FREQ*mag_PSD),axis=1) / np.sum(mag_PSD,axis=1)

# Autocorrelation Calculation
acc_acr = np.apply_along_axis(autocorr,1,acc_MAG)
print(acc_acr.shape)

acc_features = np.stack((acc_mean,acc_std,acc_max,acc_min,acc_PSDmax,acc_entropy,acc_fc),axis=1)
mag_features = np.stack((mag_mean,mag_std,mag_max,mag_min,mag_PSDmax,mag_entropy,mag_fc),axis=1)
gyr_features = np.stack((gyr_mean,gyr_std,gyr_max,gyr_min),axis=1)

X = np.concatenate([acc_features,mag_features,gyr_features],axis=1)

print("X shape: ",X.shape)
print("y shape: ",y.shape)

print("Feature Extraction Done")

(196072, 500)
X shape:  (196072, 18)
y shape:  (196072,)
Feature Extraction Done


In [4]:
# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = [
    "Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
    "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
    "Naive Bayes", "QDA"
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=1337)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

accuracies = []
for model_name, model in tqdm(zip(names, classifiers)):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f'Acc of {model_name}: {accuracy}')

plt.bar(names, accuracies)
plt.tight_layout()
plt.show()

0it [00:00, ?it/s]

Acc of Nearest Neighbors: 0.6532053659742829
Acc of Linear SVM: 0.6275500741839762


#### License for [parts of this code](https://github.com/farhanfuadabir/SHL-2020/blob/master/LICENSE)

MIT License

Copyright (c) 2020 Farhan Fuad

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the 'Software'), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED 'AS IS', WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.